# SC DOT Data and Zillow Home Sales - How has traffic in South Carolina changed?

One of my least favorite parts of the day is my daily commute. Anecdotally, I feel as if traffic has worsened/increased in South Carolina since I moved here in 2007, and it's worsening at an increasing rate every year.
I stumbled across some data on the SC-DOT GIS page that I thought might help confirm or deny my suspicion.
Here's where the data can be downloaded - https://www.scdot.org/travel/travel-mappinggis.aspx.
The data in question that I'm examining are Annual Average Daily Traffic counts from the years 2009 - 2018.
These data are recorded at the level of StationID and RouteIdentifier with additional information as well, including Latitude and Longitude of the recording Station.
I downloaded the .zip files for each separately and then renamed them and put them into one folder - shp_files.
The data come in both .shp and .dbf (Xbase) format.
More about these file types here:
TODO

SCDOT has some interactive ArcGIS maps with these data points plotted already - http://scdot.maps.arcgis.com/apps/MinimalGallery/index.html?appid=7420aa1f39d84400a6d7e8cdaacc89cd

However, these plots don't fully convey (to me) the true amount of traffic in SC, as all station points are plotted as little cars with no information about AADT, nor the change in traffic patterns year over year.
Nor do they address any underlying causes of what may be driving potential traffic pattern changes.

Tangentially, SCDOT does provide a wealth of other data for citizens to browse, some of which look quite interesting.
http://scdot.maps.arcgis.com/apps/MinimalGallery/index.html?appid=e8ace63de0e6423394d04c9c091e893b#
I am particularly interested in how the "South Carolina Roads by Pavement Status" dataset folds into the questions at hand here, but that goes beyond the scope of this post. Perhaps to be addressed later.

Getting back on topic, one of the most intuitive drivers of change in traffic patterns could be popluation growth/decline in the areas nearby. I chose to use a data set from Zillow - https://www.zillow.com/research/data/ - that details Monthly Home Sales by ZipCode as a proxy to population growth per Zip Code.

Finally, I also downloaded a data set from OpenDataSoft - https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/ - that cross references ZipCode to latitude and longitude. I only downloaded data for South Carolina.


### Questions to Answer:

1) What do the data look like and what does it take to get them in a unified format?

2) Has overall traffic increased in SC over the past 10 years?

3) Are there are any areas in SC that show more aggressive traffic growth?

4) Does number of home sales in Zip Codes within a certain radius of a station (e.g. 50 km) have any bearing on traffic numbers the next or same year?

In [35]:
# import required libraries

import pandas as pd
import numpy as np
from simpledbf import Dbf5
import os
import matplotlib.pyplot as plt

In [36]:
# read GIS dbf data into dataframes, one file for each year between 2009 and 2018
shp_dfs = {}
for root, dirs, files in os.walk("./shp_files"):
    for file in files:
        if file.endswith(".dbf"):
            # print(file.split('.')[0])
            dbf = Dbf5(os.path.join(root, file))
            df = dbf.to_dataframe()
            shp_dfs[file.split('.')[0]] = df

In [37]:
# give an ol look see at the most recent year
shp_dfs['2018'].head()

,CountyNumb,RouteTypeN,RouteType1,RouteNumbe,MeterMileP,BeginMileP,EndMilePoi,StationNum,Termini,FactoredAA,FactoredA1,MapLRS,Status1,ID1,RouteAuxil,CountyName,Long,Lat
0,1.0,2.0,US,178.0,1.196,0.00,3.82,101.0,County Line - ANDERSON TO S- 166 (DRAKE RD),4300.0,2018.0,01020017800E,NaN,1,NaN,ABBEVILLE,-82.38521,34.41979
1,1.0,2.0,US,178.0,4.345,3.82,4.91,103.0,S- 166 (DRAKE RD) TO SC 184 (MAIN ST W),4600.0,2018.0,01020017800E,NaN,2,NaN,ABBEVILLE,-82.35324,34.38344
2,1.0,2.0,US,178.0,5.633,4.91,7.28,105.0,SC 184 (MAIN ST W) TO County Line - GREENWOOD,3600.0,2018.0,01020017800E,NaN,3,NaN,ABBEVILLE,-82.33765,34.37099
3,1.0,4.0,SC,20.0,0.074,0.00,0.18,109.0,"SC 203 (WASHINGTON ST), L- 20, L- 980 TO SC 71...",4900.0,2018.0,01040002000E,NaN,4,NaN,ABBEVILLE,-82.38017,34.17889
4,1.0,4.0,SC,20.0,0.439,0.18,0.45,111.0,SC 71 (N MAIN ST) TO L- 170 (RICHEY ST),2200.0,2018.0,01040002000E,NaN,5,NaN,ABBEVILLE,-82.38114,34.18369


Just by eyeballing, it can be surmised what each field name means, along with the helpful data dictionary supplied by SCDOT.
For example "FactorerA1" is clearly the year of this particular dataset, while "Long" and "Lat" hold the longitude and latitude of the datapoint.

Next, we'll check to see if the column names match across all the dataframes we have, one for each year.

In [38]:
# check if the columns in the dfs match - per df, convert columns to sets. Check the intersection of all sets.
col_sets = map(lambda x: set(x.columns), shp_dfs.values())

In [39]:
# unpack the list of column sets into set.intersection, which returns common elements in set
common_cols = set.intersection(*col_sets)

In [40]:
# check to see what's common between the dfs
common_cols

{'ID1'}

Uh oh! There's only one column that is common between all of the dataframes. Let's dig in a little more.

In [41]:
# Let's check to see what the actual columns are named and how many there are
for df in shp_dfs.values():
    print(df.columns, len(df.columns))

Index(['CountyNumb', 'RouteTypeN', 'RouteType1', 'RouteNumbe', 'MeterMileP',
       'BeginMileP', 'EndMilePoi', 'StationNum', 'Termini', 'FactoredAA',
       'FactoredA1', 'MapLRS', 'Status1', 'ID1', 'RouteAuxil', 'CountyName',
       'Long', 'Lat'],
      dtype='object') 18
Index(['Station_Nu', 'Route_LRS', 'County_ID', 'Route_Type', 'Route_Numb',
       'Route_Auxi', 'Descriptio', 'Count', 'Year', 'ID1'],
      dtype='object') 10
Index(['STATION_NU', 'MILE_POINT', 'ROUTE_LRS', 'MAP_TYPE', 'LATITUDE',
       'LONGITUDE', 'COUNTY_ID', 'ROUTE_TYPE', 'ROUTE_NUMB', 'ROUTE_AUX',
       'COUNT', 'YEAR', 'DESCRIPTIO', 'ID1', 'GMRotation'],
      dtype='object') 15
Index(['CountyName', 'RouteTypeN', 'RouteNumbe', 'RouteAuxil', 'MeterMileP',
       'BegiNMileP', 'EndMilePoi', 'StationNum', 'Termini', 'FactoredAA',
       'FactoredA1', 'MapLRS', 'Status1', 'ID1', 'Latitude', 'Longitude'],
      dtype='object') 16
Index(['STATION', 'MILE_POINT', 'ROUTE_LRS', 'MAP_TYPE', 'ID1', 'LATITUDE',
      

We can immediately see that we have different numbers of columns per year, and that most of the columns are all named differently. We'll try to address that.

In [42]:
# maybe we'll get better results if we do some simple string formatting first
for df in shp_dfs.values():
    df.columns = [c.replace('_', '').lower().strip() for c in df.columns]

In [43]:
# check set intersection again
col_sets = list(map(lambda x: set(x.columns), shp_dfs.values()))
set.intersection(*col_sets)

{'id1'}

Still only one column that's the same! Time to do some brute-force mapping.

In [44]:
# eye all the column names
for year, df in shp_dfs.items():
    print(year, df.columns, len(df.columns))

2018 Index(['countynumb', 'routetypen', 'routetype1', 'routenumbe', 'metermilep',
       'beginmilep', 'endmilepoi', 'stationnum', 'termini', 'factoredaa',
       'factoreda1', 'maplrs', 'status1', 'id1', 'routeauxil', 'countyname',
       'long', 'lat'],
      dtype='object') 18
2013 Index(['stationnu', 'routelrs', 'countyid', 'routetype', 'routenumb',
       'routeauxi', 'descriptio', 'count', 'year', 'id1'],
      dtype='object') 10
2010 Index(['stationnu', 'milepoint', 'routelrs', 'maptype', 'latitude',
       'longitude', 'countyid', 'routetype', 'routenumb', 'routeaux', 'count',
       'year', 'descriptio', 'id1', 'gmrotation'],
      dtype='object') 15
2016 Index(['countyname', 'routetypen', 'routenumbe', 'routeauxil', 'metermilep',
       'beginmilep', 'endmilepoi', 'stationnum', 'termini', 'factoredaa',
       'factoreda1', 'maplrs', 'status1', 'id1', 'latitude', 'longitude'],
      dtype='object') 16
2015 Index(['station', 'milepoint', 'routelrs', 'maptype', 'id1', 'latitude'

In [45]:
# create a dictionary that will allow us to rename columns from key to value.
# we won't map every column - only keep a subset
col_mapping_dict = {
    **dict.fromkeys(['station', 'stationnu', 'stationnum'], 'station_id'),
    **dict.fromkeys(['milepoint', 'metermile', 'metermilep'], 'route_mile_point'),
    **dict.fromkeys(['latitude', 'lat'], 'latitude'),
    **dict.fromkeys(['longitude', 'long'], 'longitude'), 
    **dict.fromkeys(['aadtyr', 'year', 'factored1', 'factoreda1'], 'year'),
    **dict.fromkeys(['routelrs', 'maplrs'], 'route_identifier'),
    **dict.fromkeys(['termini', 'descriptio'], 'route_leg_descrip'),
    **dict.fromkeys(['beginmilep', 'beginmile'], 'route_leg_beginmile'),
    **dict.fromkeys(['endmilepo', 'endmilepoi'], 'route_leg_endmile'),
    **dict.fromkeys(['routetype', 'rtetype', 'routetypen', 'routetype1'], 'route_type'),   # has to be a numeric column as well, some collision here
    **dict.fromkeys(['rtenum', 'rtenumb', 'routenumb', 'routenum', 'routenumbe'], 'route_number'),
    **dict.fromkeys(['county', 'countyname', 'countynam'], 'county_name'),
    **dict.fromkeys(['countyid', 'countynumb'], 'county_id'),
    **dict.fromkeys(['aadt', 'factoreda', 'count', 'factoredaa'], 'average_daily_traffic'),
    **dict.fromkeys(['id1'], 'row_number')
}

In [46]:
# rename columns as per mapping dict
shp_dfs_renamed = {year: df.rename(columns=col_mapping_dict) for year, df in shp_dfs.items()}
# drop columns not mapped
#shp_dfs_renamed = {year: df.drop([c for c in df.columns if c not in col_mapping_dict.values()], axis=1) for year, df in shp_dfs_renamed.items()}
# drop any duplicated columns
shp_dfs_renamed = {year: df.loc[:, ~df.columns.duplicated()] for year, df in shp_dfs_renamed.items()}

In [47]:
# still some collision - column name can mean different things in different years.
# drop some columns that still aren't right
shp_dfs_renamed['2009'].drop(['county_name', 'route_type'], axis=1, inplace=True)
shp_dfs_renamed['2012'].drop(['county_name', 'route_type'], axis=1, inplace=True)
shp_dfs_renamed['2013'].drop('route_type', axis=1, inplace=True)
shp_dfs_renamed['2017'].drop('county_name', axis=1, inplace=True)
shp_dfs_renamed['2018'].drop('route_type', axis=1, inplace=True)

OK - columns should now all be correctly named across years. We also dropped a subset of the columns, only keeping a subset.
Let's now overwrite invalid route_types in the dfs.

In [48]:
# check to see how many NaN route types there are
for year, df in shp_dfs_renamed.items():
    if 'route_type' in df.columns:
        print(year, df.route_type.isna().sum())

2010 0
2016 0
2015 0
2017 0
2011 0
2014 0


In [49]:
# eyeball it first
# from data dictionary supplied by SC DOT
possible_route_types = ['I',
    'US',
    'SC',
    'S',
    'D',
    'R',
    'RS'
]

# check what the invalids are
for year, df in shp_dfs_renamed.items():
    if 'route_type' in df.columns:
        print(year, df.loc[~df.route_type.str.replace('-','').isin(possible_route_types)].groupby('route_type').size())


2010 route_type
L-    620
dtype: int64
2016 route_type
L-    735
dtype: int64
2015 route_type
L-    607
dtype: int64
2017 route_type
L    656
dtype: int64
2011 route_type
L-    1356
dtype: int64
2014 route_type
L-    707
dtype: int64


In [53]:
# replace invalids with np.nan
# then, fill the nans by group
for year, df in shp_dfs_renamed.items():
    if 'route_type' in df.columns:
        df.loc[~df.route_type.str.replace('-','').isin(possible_route_types), 'route_type'] = np.nan
        df['route_type'] = df.groupby(['station_id', 'route_identifier', 'route_number']).route_type.ffill().bfill()

for year, df in shp_dfs_renamed.items():
    if 'route_type' in df.columns and 'latitude' in df.columns:
        df.loc[~df.route_type.str.replace('-','').isin(possible_route_types), 'route_type'] = np.nan
        df['route_type'] = df.groupby(['station_id', 'route_identifier', 'latitude', 'longitude']).route_type.ffill().bfill()

In [54]:
# check to see how many NaN route types there are
for year, df in shp_dfs_renamed.items():
    if 'route_type' in df.columns:
        print(year, df.route_type.isna().sum())

2010 0
2016 25
2015 0
2017 656
2011 0
2014 0


In [55]:
# drop duplicated rows in dataframes
# some rows are only unique based on row_number (id1) and id2 fields
for year, df in shp_dfs_renamed.items():
    df = df.drop('row_number', axis=1)
    if 'id2' in df.columns:
        df = df.drop('id2', axis=1)
    df = df.drop_duplicates()
    shp_dfs_renamed[year] = df

Eyeballing the columns previously, we could see that we don't have latitude or longitude data fields in the 2017 and 2013 dataframes. This will cause us problems later on, so we'll address it by filling those missing columns in with data from other dataframes.

In [56]:
# take care of the 2017 dataframe first.
# update the 2017 dataframe with data from the 2018 dataframe
temp = shp_dfs_renamed['2018']. \
    set_index([
        'route_identifier',
        'station_id',
        'route_number',
        'route_leg_beginmile',
        'route_leg_endmile']
    )

twentyseventeen = shp_dfs_renamed['2017']. \
    set_index([
        'route_identifier',
        'station_id',
        'route_number',
        'route_leg_beginmile',
        'route_leg_endmile']
    )

twentyseventeen['latitude'] = np.nan
twentyseventeen['longitude'] = np.nan
twentyseventeen['county_name'] = np.nan

# use pandas update method, which uses the index to determine which rows to update. Overwrite=True means all values 
# will be overwritten in specified columns, not just nans.
twentyseventeen.update(temp[['latitude', 'longitude', 'route_leg_descrip', 'county_name']], overwrite=True)

In [57]:
# fill in any remaining gaps in 2017 data we can by updating with 2016 dataframe, not overwriting non-nans this time.
temp = shp_dfs_renamed['2016']. \
    set_index([
        'route_identifier',
        'station_id',
        'route_number',
        'route_leg_beginmile',
        'route_leg_endmile'
        ]
    )

twentyseventeen.update(temp[['latitude', 'longitude', 'route_leg_descrip', 'county_name']], overwrite=False)

In [58]:
# reassign back
shp_dfs_renamed['2017'] = twentyseventeen.reset_index()

The 2017 df is now updated (as best as possible) with Latitude and Longitude points - this will allow us to effectively plot and calculate what we want to see later on.

Now, on to the 2013 dataframe.
This dataframe presents a unique challenge because it lacks many of the distinguishing fields that would make filling in missing data easy. For example, we are missing the route_leg beginning and end mile points that we used to update the 2017 df.

In [88]:
shp_dfs_renamed['2013'].head(15)

,station_id,route_identifier,county_id,route_number,routeauxi,route_leg_descrip,average_daily_traffic,year
0,100.0,01040018500N,1.0,185.0,0.0,County Line - ANDERSON TO SC 20,1050.0,2013.0
1,101.0,01020017800E,1.0,178.0,0.0,County Line - ANDERSON TO S- 166,4600.0,2013.0
2,103.0,01020017800E,1.0,178.0,0.0,S- 166 TO SC 184,4800.0,2013.0
3,105.0,01020017800E,1.0,178.0,0.0,SC 184 TO County Line - GREENWOOD,3500.0,2013.0
4,107.0,01090002000E,1.0,20.0,0.0,"SC 72 TO SC 20, SC 203",5100.0,2013.0
5,109.0,01040002000E,1.0,20.0,0.0,"SC 203, L- 20, L- 980 TO SC 71",6200.0,2013.0
6,111.0,01040002000E,1.0,20.0,0.0,SC 71 TO L- 938,2300.0,2013.0
7,113.0,01040002000E,1.0,20.0,0.0,L- 938 TO S- 32,2200.0,2013.0
8,115.0,01040002000E,1.0,20.0,0.0,S- 32 TO S- 215,2100.0,2013.0
9,117.0,01040002000E,1.0,20.0,0.0,S- 215 TO S- 182,2200.0,2013.0


Luckily, it would appear that the 2014 and 2013 dataframes have a similar naming convention for route_leg_descrip. Combined with station_id, route_identifier, and route_number, we should be able to get a pretty good match for data filling.
We'll check to see if we have any groups larger than 1 within that grouping schema. If we do, we'll have to take care of those, as the pandas update method cannot handle duplicate multi-index entries

In [89]:
s = shp_dfs_renamed['2014'].groupby([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]).size()

ss = shp_dfs_renamed['2013'].groupby([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]).size()

# check where the size of the groups > 1
print(s[s>1])
print(ss[ss>1])

# eyeball the data
shp_dfs_renamed['2013'].loc[shp_dfs_renamed['2013'].route_leg_descrip == 'I- 85 TO S- 1164']
#shp_dfs_renamed['2012'].loc[shp_dfs_renamed['2012'].route_leg_descrip == 'County Line - YORK TO S- 121']

station_id  route_identifier  route_number  route_leg_descrip           
331.0       11070020900N      209.0         County Line - YORK TO S- 121    2
433.0       23070000500N      5.0           SC 20 TO L- 10                  2
451.0       10070001300N      13.0          US 52 TO AIR FORCE BASE ROAD    2
469.0       43090036400E      364.0         S- 537, L- 215 TO SC 441        2
749.0       39070010000E      100.0         L- 1512 TO S- 143               2
dtype: int64
station_id  route_identifier  route_number  route_leg_descrip
553.0       42070064500N      645.0         S- 736 TO S- 60      2
633.0       23090032600E      326.0         I- 85 TO S- 1164     2
923.0       42070176700N      1767.0        SC 296 TO US 176     2
dtype: int64


,station_id,route_identifier,county_id,route_number,routeauxi,route_leg_descrip,average_daily_traffic,year
4842,633.0,23090032600E,23.0,326.0,0.0,I- 85 TO S- 1164,4300.0,2013.0
4843,633.0,23090032600E,23.0,326.0,0.0,I- 85 TO S- 1164,4100.0,2013.0


In [90]:
# after eyeballing the 3 duplicates present in the 2013 data, the average daily traffic numbers are very close. Since 
# there are no other distinguishing features in the 2013 dataset, we will eliminate these 3 duplicated by averaging the 
# average daily traffic numbers across the grouping schema

twentythirteen = shp_dfs_renamed['2013'].copy(deep=True)
twentythirteen['average_daily_traffic'] = twentythirteen.\
    groupby([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]
    ). \
    average_daily_traffic. \
    transform('mean')


In [91]:
# drop the duplicates now that average daily traffic is the same across the group
twentythirteen = twentythirteen.drop_duplicates()

In [92]:
# check group sizes again
s = shp_dfs_renamed['2014'].groupby([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]).size()

ss = twentythirteen.groupby([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]).size()

print(s[s>1])
print(ss[ss>1])

# check to see if the average daily traffic was averaged
twentythirteen.loc[twentythirteen.route_leg_descrip == 'I- 85 TO S- 1164']

station_id  route_identifier  route_number  route_leg_descrip           
331.0       11070020900N      209.0         County Line - YORK TO S- 121    2
433.0       23070000500N      5.0           SC 20 TO L- 10                  2
451.0       10070001300N      13.0          US 52 TO AIR FORCE BASE ROAD    2
469.0       43090036400E      364.0         S- 537, L- 215 TO SC 441        2
749.0       39070010000E      100.0         L- 1512 TO S- 143               2
dtype: int64
Series([], dtype: int64)


,station_id,route_identifier,county_id,route_number,routeauxi,route_leg_descrip,average_daily_traffic,year
4842,633.0,23090032600E,23.0,326.0,0.0,I- 85 TO S- 1164,4200.0,2013.0


In [93]:
# eyeball a 2014 dupe
shp_dfs_renamed['2014'].loc[shp_dfs_renamed['2014'].route_leg_descrip == 'County Line - YORK TO S- 121']

,station_id,route_mile_point,route_identifier,maptype,latitude,longitude,county_name,route_type,route_number,aux,atrnum,map,route_leg_descrip,year,average_daily_traffic,gmrotation
2376,331.0,0.543,11070020900N,COUNTY,35:2:59.839,-81:28:48.254,Cherokee,S-,209.0,NaN,NaN,Y,County Line - YORK TO S- 121,2014.0,225,0.000002
2377,331.0,0.566,11070020900N,COUNTY,35:3:0.868,-81:28:48.957,Cherokee,S-,209.0,NaN,NaN,Y,County Line - YORK TO S- 121,2014.0,225,0.000002


For our update df (2014), we will eliminate the 5 duplicate sets by taking the first row from each set - the reasoning
here is that the duplicates in the 2014 dataset on route_leg_descrip arise from different mile markers along
the same route_leg. The lat/long datapoints for two points along the same route_leg_descrip should be very close.
Therefore, we will choose one of the lat/long datapoints arbitrarily.

In [94]:
# take the first row from each group
update_temp = shp_dfs_renamed['2014'].groupby([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]).head(1)

# verify that we have no dupes
s = update_temp.groupby([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]).size()

print(s[s>1])

Series([], dtype: int64)


In [95]:
# update the twentythirteen dataframe with the 2014 update dataframe
update_temp = update_temp. \
    set_index([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip' 
        ]
    )

twentythirteen = twentythirteen .\
    set_index([
        'station_id',
        'route_identifier',
        'route_number',
        'route_leg_descrip'
        ]
    )

twentythirteen['longitude'] = np.nan
twentythirteen['latitude'] = np.nan

twentythirteen.update(update_temp[['latitude', 'longitude']], overwrite=True)

In [87]:
twentythirteen.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11286 entries, 0 to 11285
Data columns (total 10 columns):
station_id               11286 non-null float64
route_identifier         11286 non-null object
route_number             11286 non-null float64
route_leg_descrip        11286 non-null object
county_id                11286 non-null float64
routeauxi                11286 non-null float64
average_daily_traffic    11286 non-null float64
year                     11286 non-null float64
longitude                10877 non-null object
latitude                 10877 non-null object
dtypes: float64(6), object(4)
memory usage: 881.8+ KB


We have successfully filled in lat/long points for 10877 rows in the 2013 df.

In [96]:
# reassign back
shp_dfs_renamed['2013'] = twentythirteen.reset_index()

You may have noticed that some dataframes have the lat/long as colon-separate strings, while other dataframes have them as floats.
We'll have to convert strings to floats for plotting.

In [125]:
def convert_lat_long(series):
    """Convert a series of lat or long strings into floating point representation.
    
    Args:
        series (pandas Series): a series of strings of lat/long, delimited by a colon.
    Returns:
        converted (pandas Series): a series of floating points of lat/long, rounded to 5 decimal places
    
    """
    # expand=True turns the delimited series into a dataframe with multiple columns (rather than list of splits)
    split = series.str.split(':', expand=True)
    # compute the floating point representation
    converted = split[0].astype('float') + split[1].astype('float') / 60 + split[2].astype('float') / (60*60)
    
    return converted.round(5)


In [156]:
shp_dfs_renamed['2013'].latitude.dtype

for year, df in shp_dfs_renamed.items():
    if df.latitude.dtype == 'O' and df.latitude.str.contains(':').any():
        df['latitude'] = convert_lat_long(df.latitude)
        df['longitude'] = convert_lat_long(df.longitude)
    else:
        df['latitude'] = df.latitude.astype('float')
        df['longitude'] = df.longitude.astype('float')

In [192]:
s = shp_dfs_renamed['2014'].groupby([
    shp_dfs_renamed['2014'].latitude.round(2),
    shp_dfs_renamed['2014'].longitude.round(2),
 'station_id', 'route_identifier', 'route_number', 'route_mile_point', 'route_leg_descrip']).size()
s[s>1]

latitude  longitude  station_id  route_identifier  route_number  route_mile_point  route_leg_descrip           
32.90     -79.98     451.0       10070001300N      13.0          3.229             US 52 TO AIR FORCE BASE ROAD    2
33.99     -79.53     469.0       43090036400E      364.0         2.877             S- 537, L- 215 TO SC 441        2
34.83     -81.59     433.0       23070000500N      5.0           2.669             SC 20 TO L- 10                  2
34.97     -81.13     749.0       39070010000E      100.0         3.390             L- 1512 TO S- 143               2
dtype: int64

In [195]:
shp_dfs_renamed['2014'][shp_dfs_renamed['2014'].route_leg_descrip == 'L- 1512 TO S- 143']
shp_dfs['2014'].iloc[8934:8936]

,station,milepoint,routelrs,maptype,id1,latitude,longitude,countyname,routetype,rtenum,aux,atrnum,map,termini,year,count,id2,gmrotation
8934,749.0,3.39,39070010000E,COUNTY,2532,34:57:57.006,-82:52:4.314,Pickens,S-,100.0,NaN,NaN,Y,L- 1512 TO S- 143,2014.0,550,8935,8.004128e-07
8935,749.0,3.39,39070010000E,COUNTY,2532,34:57:57.006,-82:52:4.314,Pickens,L-,100.0,NaN,NaN,Y,L- 1512 TO S- 143,2014.0,600,8936,8.004128e-07


In [181]:
shp_dfs_renamed['2012'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11275 entries, 0 to 11274
Data columns (total 19 columns):
station_id               11275 non-null float64
route_mile_point         11261 non-null float64
route_identifier         11275 non-null object
maptype                  11273 non-null object
latitude                 11265 non-null float64
longitude                11265 non-null float64
county_id                11275 non-null int64
f2                       11275 non-null object
station1                 11275 non-null float64
route_number             11275 non-null float64
rteaux                   11275 non-null float64
bmp                      11275 non-null float64
emp                      11275 non-null float64
route_leg_descrip        11275 non-null object
rtelrs                   11275 non-null object
connn                    11275 non-null object
average_daily_traffic    11275 non-null int64
year                     11275 non-null int64
gmrotation               11275 non-null

In [124]:
shp_dfs_renamed['2018'].merge(shp_dfs_renamed['2014'], how='left', on='latitude').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11672 entries, 0 to 11671
Data columns (total 30 columns):
county_id                  11672 non-null float64
route_number_x             11672 non-null float64
route_mile_point_x         11671 non-null float64
route_leg_beginmile        11672 non-null float64
route_leg_endmile          11672 non-null float64
station_id_x               11672 non-null float64
route_leg_descrip_x        11672 non-null object
average_daily_traffic_x    11603 non-null float64
year_x                     11656 non-null float64
route_identifier_x         11672 non-null object
status1                    183 non-null object
routeauxil                 335 non-null object
county_name_x              11487 non-null object
longitude_x                11489 non-null object
latitude                   11489 non-null object
station_id_y               0 non-null float64
route_mile_point_y         0 non-null float64
route_identifier_y         0 non-null object
maptype        

In [144]:
s[s>1]
#twentythirteen = shp_dfs_renamed['2013'].drop('row_number', axis=1).drop_duplicates()

station_id  route_identifier  route_number
426.0       04090713500N      7135.0          2
553.0       42070064500N      645.0           2
633.0       23090032600E      326.0           2
923.0       42070176700N      1767.0          2
dtype: int64

In [134]:
g = twentythirteen.groupby(['station_id', 'route_identifier', 'route_number']).size() 
g[g>1]

station_id  route_identifier  route_number
426.0       04090713500N      7135.0          2
553.0       42070064500N      645.0           2
633.0       23090032600E      326.0           2
923.0       42070176700N      1767.0          2
dtype: int64

In [50]:
# drop duplicates
for year, df in shp_dfs_renamed.items():
    df = df.drop_duplicates()
    shp_dfs_renamed[year] = df
        

In [51]:
# records should be unique across station_id, route_identifier, and route_number - do some verification
for year, df in shp_dfs_renamed.items():
    temp_df = df.groupby(['station_id', 'route_identifier', 'route_number']).size()
    # print rows where the size of the group is > 1 (duplicates)
    print(year,'\n', temp_df.loc[temp_df > 1.0])

2018 
 station_id  route_identifier  route_number
101.0       32020000100N      1.0             2
321.0       21070013200E      132.0           2
dtype: int64
2013 
 station_id  route_identifier  route_number
426.0       04090713500N      7135.0          2
553.0       42070064500N      645.0           2
633.0       23090032600E      326.0           2
923.0       42070176700N      1767.0          2
dtype: int64
2010 
 station_id  route_identifier  route_number
169.0       04040002400E      24.0            2
170.0       40020007600E      76.0            2
329.0       22070039100N      391.0           2
331.0       11070020900N      209.0           2
333.0       22070039100N      878.0           2
2601.0      02010052000E      520.0           2
dtype: int64
2016 
 station_id  route_identifier  route_number
101.0       32020000100N      1.0             2
dtype: int64
2015 
 Series([], dtype: int64)
2009 
 station_id  route_identifier  route_number
331.0       11070020900N      209.0       

It would appear that we have some duplicates, or we think we have duplicates. We'll define a quick function to spot check some records and get a sense of why rows are duplicated.

In [52]:
# we can spot check some of these to see what's up with the records
def check_records(year, station_id, route_identifier):
    mask = (shp_dfs_renamed[year]['station_id'] == station_id) & (shp_dfs_renamed[year]['route_identifier'] == route_identifier)
    return shp_dfs_renamed[year].loc[mask]

check_records('2014', 451.0, '10070001300N') 

,station_id,route_identifier,latitude,longitude,county_name,route_type,route_number,route_leg_descrip,year,average_daily_traffic
2054,451.0,10070001300N,32:53:53.404,-80:1:10.830,Charleston,S-,13.0,US 52 TO AIR FORCE BASE ROAD,2014.0,19500
2055,451.0,10070001300N,32:53:53.404,-80:1:10.830,Charleston,S-,13.0,US 52 TO AIR FORCE BASE ROAD,2014.0,18200


In [53]:
# check the underlying data
shp_dfs['2014'].loc[(shp_dfs['2014'].station == 451.0) & (shp_dfs['2014'].routelrs == '10070001300N')]

,station,milepoint,routelrs,maptype,id1,latitude,longitude,countyname,routetype,rtenum,aux,atrnum,map,termini,year,count,id2,gmrotation
2054,451.0,3.229,10070001300N,CHARLESTON,5653,32:53:53.404,-80:1:10.830,Charleston,S-,13.0,NaN,NaN,Y,US 52 TO AIR FORCE BASE ROAD,2014.0,19500,2055,8.170881e-07
2055,451.0,3.229,10070001300N,CHARLESTON,5653,32:53:53.404,-80:1:10.830,Charleston,L-,13.0,NaN,NaN,Y,US 52 TO AIR FORCE BASE ROAD,2014.0,18200,2056,8.170881e-07


The above two rows differ based on id2 and count

In [54]:
check_records('2009', 331.0, '11070020900N')
#shp_dfs['2010'].loc[4407:4408]

,station_id,route_identifier,latitude,longitude,route_number,average_daily_traffic,year,route_leg_descrip
2297,331.0,11070020900N,35:03:00.0546,-81:28:47.765,209.0,250.0,2009.0,York County Line TO S-121
2298,331.0,11070020900N,35:03:00.8678,-81:28:48.9568,209.0,250.0,2009.0,York County Line TO S-121


In [56]:
shp_dfs['2009'].loc[(shp_dfs['2009'].station == 331.0) & (shp_dfs['2009'].routelrs == '11070020900N')]

,station,milepoint,routelrs,maptype,id1,latitude,longitude,countyid,atrnum,county,stationnu,routetype,routenumb,aux,count,year,descriptio,gmrotation
2297,331.0,0.566,11070020900N,COUNTY,4444,35:03:00.0546,-81:28:47.765,11,NaN,11.0,331.0,7.0,209.0,0.0,250.0,2009.0,York County Line TO S-121,0.000002
2298,331.0,0.566,11070020900N,COUNTY,7134,35:03:00.8678,-81:28:48.9568,11,NaN,11.0,331.0,7.0,209.0,0.0,250.0,2009.0,York County Line TO S-121,0.000002


In [58]:
check_records('2016', 101.0, '32020000100N')

,county_name,route_type,route_number,station_id,route_leg_descrip,average_daily_traffic,year,route_identifier,latitude,longitude
6377,LEXINGTON,US,1.0,101.0,County Line - SALUDA TO S- 50 (N RIDGELL ST),2700.0,2016.0,32020000100N,33.8975758564561,-81.5616927735785
6396,LEXINGTON,US,1.0,101.0,US 378 (MEETING ST) (LEXINGTON) TO US 21 (HUG...,27900.0,2016.0,32020000100N,33.8981999185585,-81.5602154592896


In [62]:
shp_dfs['2016'].loc[(shp_dfs['2016'].stationnum == 101.0) & (shp_dfs['2016'].maplrs == '32020000100N')]

,countyname,routetypen,routenumbe,routeauxil,metermilep,beginmilep,endmilepoi,stationnum,termini,factoredaa,factoreda1,maplrs,status1,id1,latitude,longitude
6377,LEXINGTON,US,1.0,NaN,0.047,0.00,0.72,101.0,County Line - SALUDA TO S- 50 (N RIDGELL ST),2700.0,2016.0,32020000100N,NaN,6378,33.8975758564561,-81.5616927735785
6396,LEXINGTON,US,1.0,NaN,0.148,31.22,31.40,101.0,US 378 (MEETING ST) (LEXINGTON) TO US 21 (HUG...,27900.0,2016.0,32020000100N,NaN,6397,33.8981999185585,-81.5602154592896


In [73]:
# eyeballing the records reveals that they are duplicated rows. we'll take the first from every group
# head(1) will return the first row per group
for year, df in shp_dfs_renamed.items():
    temp_df = df.groupby(['station_id', 'route_identifier', 'route_number']).head(1)
    shp_dfs_renamed[year] = temp_df

Eyeballing a few sets of records leads me to believe that we are in fact dealing with duplicate rows. Most are only unique based on the row number. We'll drop our duplicates to make sure they don't sully the dataset.

Check for duplicates again.

In [74]:
#check for dupes again
for year, df in shp_dfs_renamed.items():
    temp_df = df.groupby(['station_id', 'route_identifier', 'route_number']).size()
    print(year,'\n', temp_df.loc[temp_df > 1.0])

2018 
 Series([], dtype: int64)
2013 
 Series([], dtype: int64)
2010 
 Series([], dtype: int64)
2016 
 Series([], dtype: int64)
2015 
 Series([], dtype: int64)
2009 
 Series([], dtype: int64)
2017 
 Series([], dtype: int64)
2012 
 Series([], dtype: int64)
2011 
 Series([], dtype: int64)
2014 
 Series([], dtype: int64)


In [75]:
# verify we took the first row in the duplicated group, and that we only have one row returning.
check_records('2011', 2520.0, '08010052600E')

,station_id,average_daily_traffic,route_leg_descrip,year,route_type,route_number,route_identifier,latitude,longitude
1675,2520.0,53400.0,SEVEN FARMS RD TO S- 97 (CHARLESTON),2011.0,I-,526.0,08010052600E,32:51:39.5474,-79:53:52.4178


At this point we've realized that the data can very quite widely from year to year. In order to clean the data up a little bit more, we will try to standardize the data across years. To accomplish this we'll use pandas' update method - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.update.html - and overwrite the values in every year with our standardized values. Because the 2017 and 2018 dataframes don't have all the columns that we're interested in looking at, we'll use the 2016 dataframe to update/overwrite values in a specific subset of columns across all years.

In [76]:
# set index of all dfs to the unique identifiers
for year, df in shp_dfs_renamed.items():
    temp_df = df.set_index(['station_id', 'route_identifier', 'route_number'])
    shp_dfs_renamed[year] = temp_df

In [77]:
# fill missing year value in any dfs
for year, df in shp_dfs_renamed.items():
    df['year'] = df.year.fillna(year)

In [78]:
# 2018 and 2017 dfs don't have all the columns, so use the 2016 df to standardize fields
# check how many nas in each columns in the 2016 df
shp_dfs_renamed['2016'].isna().sum()

county_name                0
route_type               726
route_leg_descrip          0
average_daily_traffic      0
year                       0
latitude                 179
longitude                179
dtype: int64

In [79]:
# we have some NaNs for lat/long in the 2016 df, so we'll try and reduce those as much as we can.
# update 2016 latitudes/longitudes with 2018 latitudes/longitudes where stationid, route_id, and route_number match (index of each df)

shp_dfs_renamed['2016'].update(shp_dfs_renamed['2018'][['latitude', 'longitude']], overwrite=False)

# still some nulls - try the 2015 df
shp_dfs_renamed['2016'].update(shp_dfs_renamed['2015'][['latitude', 'longitude']], overwrite=False)

In [80]:
# check nas again
shp_dfs_renamed['2016'].isna().sum()

county_name                0
route_type               726
route_leg_descrip          0
average_daily_traffic      0
year                       0
latitude                 153
longitude                153
dtype: int64

In [81]:
# 2018 and 2017 dfs don't have all the columns, so use the 2016 df to standardize fields
# we're standardizing data across the five following columns - DON'T want to overwrite AADT.
cols_to_update = ['route_type', 'route_leg_descrip', 'latitude', 'longitude', 'county_name']
update_df = shp_dfs_renamed['2016'][cols_to_update]

for year, df in shp_dfs_renamed.items():
    df.update(update_df, overwrite=True)

At this point we should be ready to combine all the dataframes together.

In [82]:
# stack all the data frames together
traffic_df = pd.concat(shp_dfs_renamed.values(), sort=True, axis=0).reset_index()

In [83]:
# eyeball the data 
traffic_df.head()

,station_id,route_identifier,route_number,average_daily_traffic,county_name,latitude,longitude,route_leg_descrip,route_type,year
0,101.0,01020017800E,178.0,4300.0,ABBEVILLE,34.41979,-82.38521,County Line - ANDERSON TO S- 166 (DRAKE RD),NaN,2018
1,103.0,01020017800E,178.0,4600.0,ABBEVILLE,34.38344,-82.35325,S- 166 (DRAKE RD) TO SC 184 (N MAIN ST),NaN,2018
2,105.0,01020017800E,178.0,3600.0,ABBEVILLE,34.37099,-82.33765,SC 184 (N MAIN ST) TO County Line - GREENWOOD,NaN,2018
3,109.0,01040002000E,20.0,4900.0,ABBEVILLE,34.17888,-82.38016,"SC 203 (WASHINGTON ST), L- 20, L- 980 TO SC 71",NaN,2018
4,111.0,01040002000E,20.0,2200.0,ABBEVILLE,34.18359,-82.38115,SC 71 TO L- 170,NaN,2018


Eyeballing the head indicates that there are STILL NaNs in the data - the index of our update df (stationid, routeid, routenumber) did not overlap every possible combination in the data across the years. We can handle these NaNs with a similar operation - grouping the data by our unique identifers and then filling within groups

In [85]:
# now fill in cols that are still na by unique id - NOT the average traffic column (main data we care about)
# fill nas by group
cols_to_fill = ['county_name', 'latitude', 'longitude', 'route_leg_descrip', 'route_type']
for col in cols_to_fill:
    traffic_df[col] = traffic_df.groupby(['station_id', 'route_identifier', 'route_number'])[col].ffill().bfill()

In [86]:
# check for any nas remaining
traffic_df.isna().sum()

station_id                0
route_identifier          0
route_number              0
average_daily_traffic    69
county_name               0
latitude                  0
longitude                 0
route_leg_descrip         0
route_type                0
year                      0
dtype: int64

In [88]:
# drop remaining nas - no data!
traffic_df = traffic_df.dropna()

In [115]:
# do some test plotting
test_plot = traffic_df.loc[traffic_df.year == 2018].sample(1000)
x_plot = test_plot.longitude.values
y_plot = test_plot.latitude.values
cmap = plt.get_cmap('jet')
c = np.log(test_plot.average_daily_traffic)
s = np.log(test_plot.average_daily_traffic)

# plt.scatter(x_plot, y_plot, alpha=0.15, s=s, c=c, cmap=cmap)
plt.scatter(x_plot, y_plot)
#plt.scatter([1,2,3], [2,3,4])

Error in callback <function flush_figures at 0x7fe1857d8320> (for post_execute):


KeyboardInterrupt: 

In [116]:
# can't plot all lats and longs because the lats/longs are strings.
test_plot.tail(165)

,station_id,route_identifier,route_number,average_daily_traffic,county_name,latitude,longitude,route_leg_descrip,route_type,year
1547,285.0,08070009700N,97.0,225.0,BERKELEY,33.23964,-79.89429,US 17 ALT (N HWY 17A) TO SC 402 (HWY 402),SC,2018
861,539.0,04070024600E,246.0,1400.0,ANDERSON,34.51393,-82.57251,S- 245 (BLUE RIDGE AV) TO US 29 (HIGHWAY 29 N),SC,2018
11325,696.0,39090002400E,24.0,6400.0,PICKENS,34.67119,-82.83522,L- 320 (PERIMETER RD) TO US 76 (ANDERSON HWY),SC,2018
2399,209.0,12040000907S,9.0,7300.0,CHESTER,34.70476,-81.20483,US 321 BUS (COLUMBIA ST) TO S- 103 (HARRIS ST),SC,2018
4501,273.0,23040041400E,414.0,2500.0,GREENVILLE,35.04325,-82.43646,US 276 (GEER HWY) TO US 25 (N 25 HWY),SC,2018
...,...,...,...,...,...,...,...,...,...,...
8677,105.0,40020000100N,1.0,26400.0,RICHLAND,34.04057,-80.99772,SC 16 (N BELTLINE BLVD) TO S- 289 CON (PINE BE...,SC,2018
9257,2309.0,42010008500N,85.0,96500.0,SPARTANBURG,34.90371,-82.12373,SC 101 (HIGHWAY 101 S) TO SC 290 (E MAIN ST),SC,2018
1896,314.0,10070005100N,51.0,13600.0,CHARLESTON,32.82748,-79.81605,SC 517 (ISLE OF PALMS CONNECTOR) TO S- 921 (SI...,SC,2018
4186,505.0,21070092500N,925.0,3000.0,FLORENCE,34.21543,-79.70426,S- 24 TO S- 13 (MOORE ST),SC,2018


In [33]:
for year, df in shp_dfs_renamed.items():
    try:
        print(year, df.latitude.str.contains(':').sum())
    except:
        pass
update_df.latitude.str.contains(':').sum()

2018 0
2010 404
2016 0
2015 0
2009 442
2012 359
2011 371
2014 163


0

In [34]:
traffic_df.loc[traffic_df.latitude.str.contains(':')]

,station_id,route_identifier,route_number,average_daily_traffic,county_name,latitude,longitude,route_leg_descrip,route_type,year
22931,107.0,01040002000E,20.0,5800.0,ABBEVILLE,34:10:30.277,-82:22:34.9755,S.C. 72 TO Pickens Street,SC,2010
22962,170.0,01040007200E,579.0,1750.0,ABBEVILLE,34:05:28.4781,-82:35:57.0661,SC-72 to SC-72,S-,2010
22964,173.0,01070024000E,72.0,3400.0,ABBEVILLE,34:07:42.3243,-82:26:00.7028,S-185 TO S.C. 28,SC,2010
23019,262.0,01070007300N,73.0,400.0,ABBEVILLE,34:27:19.8048,-82:21:05.8571,Anderson County Line to SC-252,S-,2010
23046,315.0,01070014700N,147.0,100.0,ABBEVILLE,34:22:54.3097,-82:25:16.6313,S-89 TO S-321,S-,2010
...,...,...,...,...,...,...,...,...,...,...
112437,469.0,43090036400E,364.0,5500.0,Sumter,33:59:10.318,-80:28:20.795,"S- 537, L- 215 TO SC 441",S-,2014
112462,515.0,43070103400E,1428.0,4000.0,Sumter,33:57:29.149,-80:22:42.384,US 521 TO S- 269,S-,2014
112599,743.0,43070014500N,586.0,1450.0,Sumter,33:55:59.163,-80:20:2.554,"S- 234 TO S- 123, L- 585",S-,2014
112649,158.0,44040005600W,56.0,800.0,Union,34:36:0.780,-81:50:58.284,County Line - LAURENS TO County Line - SPARTAN...,SC,2014


In [36]:
traffic_df.loc[traffic_df.latitude.str.contains(':'), 'latitude'] = traffic_df.loc[traffic_df.latitude.str.contains(':'), 'latitude'].apply(convert_lat_long) 

In [37]:
traffic_df.loc[traffic_df.longitude.str.contains(':'), 'longitude'] = traffic_df.loc[traffic_df.longitude.str.contains(':'), 'longitude'].apply(convert_lat_long) 

In [38]:
traffic_df['latitude'] = traffic_df.latitude.astype('float')

In [39]:
traffic_df['longitude'] = traffic_df.longitude.astype('float')

In [41]:
# get the pct change year over year for average daily traffic
traffic_df['traffic_yearly_pct_change'] = traffic_df \
    .sort_values(['station_id', 'route_identifier', 'route_number', 'year']) \
    .groupby(['station_id', 'route_identifier', 'route_number']) \
    .average_daily_traffic \
    .pct_change()

In [42]:
traffic_df = traffic_df.sort_values(['station_id', 'route_identifier', 'route_number', 'year'])

In [43]:
traffic_df[traffic_df['traffic_yearly_pct_change'] > 0.2]

,station_id,route_identifier,route_number,average_daily_traffic,county_name,latitude,longitude,route_leg_descrip,route_type,year,traffic_yearly_pct_change
46302,100.0,04020002900N,29.0,4000.0,ANDERSON,34.35590,-82.81412,State Line - GEORGIA TO SC 187 (HIGHWAY 187 S),US,2015,0.290323
69660,100.0,08020001702N,17.0,53100.0,BERKELEY,33.03701,-80.15366,County Line - DORCHESTER TO I- 26,US,2017,0.288835
107867,100.0,28040001200E,12.0,4100.0,KERSHAW,34.11999,-80.77968,County Line - RICHLAND TO S- 47 (FORT JACKSON RD),SC,2014,0.413793
46303,101.0,04020002900N,29.0,2100.0,ANDERSON,34.40918,-82.79075,SC 187 (HIGHWAY 187 S) TO US 29 BUS (HIGHWAY ...,US,2015,0.312500
35410,101.0,07020001700N,17.0,12400.0,BEAUFORT,32.64004,-80.85637,County Line - JASPER TO US 17 ALT (CASTLE HALL...,US,2016,0.441860
...,...,...,...,...,...,...,...,...,...,...,...
50675,2435.0,23010018500N,185.0,5000.0,GREENVILLE,34.77331,-82.44549,SC 153 (153 HWY) TO I- 85,I-,2015,0.250000
95757,2439.0,23010018500N,185.0,16700.0,GREENVILLE,34.80352,-82.42446,"US 25 (WHITE HORSE RD) TO , SC 20",I-,2011,0.590476
9271,2489.0,42010058500S,585.0,32200.0,SPARTANBURG,34.97494,-81.94188,"US 176 CO2 (N CHURCH ST), SC 9 TO US 221 (WHIT...",SC,2018,0.201493
9272,2491.0,42010058500S,585.0,31200.0,SPARTANBURG,34.97158,-81.93761,US 221 (WHITNEY RD) TO US 176 (N PINE ST),SC,2018,0.214008


### Zillow Sales data by Zip Code

In [44]:
# navigate up
os.chdir('..')

In [53]:
# read in the data
sales_df = pd.read_csv('Sale_Counts_Zip.csv')
sales_df.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,seasAdj
0,61639,10025,New York,1,NaN,NaN,NaN,NaN,NaN,NaN,...,76.0,33.0,47.0,56.0,35.0,70.0,78.0,66.0,63.0,0
1,84654,60657,Illinois,2,NaN,NaN,NaN,NaN,NaN,NaN,...,91.0,77.0,113.0,157.0,189.0,165.0,186.0,141.0,152.0,0
2,61637,10023,New York,3,NaN,NaN,NaN,NaN,NaN,NaN,...,80.0,45.0,63.0,45.0,66.0,85.0,79.0,90.0,95.0,0
3,91982,77494,Texas,4,56.0,71.0,84.0,95.0,116.0,86.0,...,86.0,112.0,186.0,218.0,200.0,204.0,245.0,226.0,NaN,0
4,84616,60614,Illinois,5,NaN,NaN,NaN,NaN,NaN,NaN,...,75.0,85.0,144.0,163.0,219.0,209.0,204.0,196.0,173.0,0


The data is very wide - sales are recording in columns. Our steps for prepping this data will be to:
1) Filter down to only South Carolina Zip Codes

2) Drop columns we don't care about

3) Convert records from wide to tall format

4) Get the sum of sales per zipcode per year

In [54]:
# filter to only SC
sales_df = sales_df.loc[sales_df.StateName == 'South Carolina']

# drop columns we don't care about and rename columns
sales_df = sales_df.drop(['RegionID', 'StateName', 'seasAdj'], axis=1)
sales_df.rename(columns={'RegionName': 'ZipCode'}, inplace=True)
sales_df = sales_df.set_index(['ZipCode', 'SizeRank'])

In [51]:
sales_df.head()

,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09
ZipCode,,,,,,,,,,,,,,,,,,,,,
29732,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.0,75.0,75.0,134.0,140.0,125.0,136.0,128.0,138.0,151.0
29072,227,63.0,89.0,75.0,70.0,76.0,65.0,68.0,47.0,42.0,...,165.0,88.0,88.0,126.0,132.0,192.0,156.0,180.0,169.0,142.0
29730,287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,119.0,71.0,60.0,91.0,95.0,116.0,111.0,87.0,100.0,107.0
29464,291,58.0,78.0,73.0,88.0,81.0,73.0,57.0,69.0,49.0,...,76.0,63.0,91.0,94.0,138.0,116.0,135.0,146.0,141.0,NaN
29681,350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,99.0,118.0,150.0,129.0,165.0,136.0,151.0,150.0,161.0


In [55]:
# stack all sales into one column
sales_df = sales_df.stack().reset_index()
sales_df.columns = ['ZipCode', 'SizeRank', 'YearMonth', 'Sales']
sales_df

,ZipCode,SizeRank,YearMonth,Sales
0,29732,180,2014-08,127.0
1,29732,180,2014-09,145.0
2,29732,180,2014-10,124.0
3,29732,180,2014-11,121.0
4,29732,180,2014-12,124.0
...,...,...,...,...
51258,820,31089,2019-05,0.0
51259,820,31089,2019-06,0.0
51260,820,31089,2019-07,0.0
51261,820,31089,2019-08,0.0


In [56]:
# extract the year out of the Sales record
sales_df['Year'] = sales_df['YearMonth'].str.split('-').str[0]
sales_df.head()

,ZipCode,SizeRank,YearMonth,Sales,Year
0,29732,180,2014-08,127.0,2014
1,29732,180,2014-09,145.0,2014
2,29732,180,2014-10,124.0,2014
3,29732,180,2014-11,121.0,2014
4,29732,180,2014-12,124.0,2014


In [57]:
# yearly sales calculated - sum over ZipCode and year
yearly_sales = sales_df.groupby(['ZipCode', 'SizeRank', 'Year']).Sales.sum()
yearly_sales.tail(15)

ZipCode  SizeRank  Year
29944    13111     2017    42.0
                   2018    28.0
                   2019     3.0
29945    12932     2008    22.0
                   2009    20.0
                   2010    20.0
                   2011    25.0
                   2012    30.0
                   2013    23.0
                   2014    23.0
                   2015    26.0
                   2016    14.0
                   2017    23.0
                   2018    17.0
                   2019     3.0
Name: Sales, dtype: float64

### Zip to Lat/Long xref

In [90]:
# read in the data
zip_xref = pd.read_csv('sc-zip-code-latitude-and-longitude.csv')

In [91]:
zip_xref.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,Unnamed: 8
0,29607,Greenville,SC,34.825592,-82.34099,-5,1,34.825592,-82.34099
1,29164,Wagener,SC,33.659078,-81.40845,-5,1,33.659078,-81.40845
2,29325,Clinton,SC,34.470115,-81.86761,-5,1,34.470115,-81.86761
3,29520,Cheraw,SC,34.688620,-79.92315,-5,1,34.688620,-79.92315
4,29615,Greenville,SC,34.866801,-82.31739,-5,1,34.866801,-82.31739


In [92]:
# merge the zip code to lat/long xref to the yearly sales data
merged_sales = yearly_sales.reset_index().merge(zip_xref, how='left', left_on='ZipCode', right_on='Zip')

In [93]:
# drop zip codes that have a length less than 5
merged_sales = merged_sales.loc[merged_sales.ZipCode.apply(lambda x: len(str(x)) >= 5)]

In [94]:
# drop columns i don't care about
merged_sales = merged_sales.drop(['Zip','Timezone', 'Daylight savings time flag', 'geopoint', 'Unnamed: 8'], axis=1)

In [95]:
merged_sales.head(25)

,ZipCode,SizeRank,Year,Sales,City,State,Latitude,Longitude
12,29001,17838,2008,10.0,Alcolu,SC,33.769930,-80.17278
13,29001,17838,2009,10.0,Alcolu,SC,33.769930,-80.17278
14,29001,17838,2010,11.0,Alcolu,SC,33.769930,-80.17278
15,29001,17838,2011,8.0,Alcolu,SC,33.769930,-80.17278
16,29001,17838,2012,9.0,Alcolu,SC,33.769930,-80.17278
17,29001,17838,2013,12.0,Alcolu,SC,33.769930,-80.17278
18,29001,17838,2014,10.0,Alcolu,SC,33.769930,-80.17278
19,29001,17838,2015,3.0,Alcolu,SC,33.769930,-80.17278
20,29001,17838,2016,14.0,Alcolu,SC,33.769930,-80.17278
21,29001,17838,2017,12.0,Alcolu,SC,33.769930,-80.17278


At this point I'd like to calculate how far each Zip Code is from each station. 
I searched online for a way to calculate the as-the-crow-flies distance between two lat/long points.
The first reference I found referred to the "haversine" formula, detailed here - https://www.movable-type.co.uk/scripts/latlong.html. 
I found a nice numpy implmentation on Stack Overflow here: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas from derricw.

In order to calculate the distance between each possible combination of points, I will be taking all unique lat/long points from traffic data and crossjoining with all unique lat/long points from zip code data. Then, I will apply the haversine formula to get the linear distance between each pair of points.

In [96]:
import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [97]:
# get unique zip to lat/long
zip_xref['Latitude'] = zip_xref.Latitude.astype('float')
zip_xref['Longitude'] = zip_xref.Longitude.astype('float')
unique_zip = zip_xref.loc[:, ['Zip', 'Latitude', 'Longitude']].drop_duplicates()

In [98]:
unique_zip

,Zip,Latitude,Longitude
0,29607,34.825592,-82.34099
1,29164,33.659078,-81.40845
2,29325,34.470115,-81.86761
3,29520,34.688620,-79.92315
4,29615,34.866801,-82.31739
...,...,...,...
549,29592,34.283207,-79.47272
550,29646,34.169781,-82.15474
551,29142,33.462378,-80.50903
552,29449,32.715745,-80.26738


In [99]:
# get the unique station/routeid/route number lat/long
unique_station = traffic_df \
    .loc[:, ['station_id', 'route_identifier', 'route_number', 'latitude', 'longitude']] \
    .drop_duplicates()

In [103]:
# create the cartesian product/crossjoin by merging on a uniform key
unique_zip['key'] = 0
unique_station['key'] = 0

station_zip_xref = unique_station.merge(unique_zip, on='key').drop('key', axis=1)
station_zip_xref.columns = [
        'station_id',
        'route_identifier',
        'route_number',
        'station_lat',
        'station_long',
        'Zip',
        'zip_lat',
        'zip_long'
    ]

station_zip_xref

,station_id,route_identifier,route_number,station_lat,station_long,Zip,zip_lat,zip_long
0,100.0,01040018500N,185.0,34.37925,-82.44926,29607,34.825592,-82.34099
1,100.0,01040018500N,185.0,34.37925,-82.44926,29164,33.659078,-81.40845
2,100.0,01040018500N,185.0,34.37925,-82.44926,29325,34.470115,-81.86761
3,100.0,01040018500N,185.0,34.37925,-82.44926,29520,34.688620,-79.92315
4,100.0,01040018500N,185.0,34.37925,-82.44926,29615,34.866801,-82.31739
...,...,...,...,...,...,...,...,...
7061833,2607.0,02010052000E,520.0,33.56019,-81.92848,29592,34.283207,-79.47272
7061834,2607.0,02010052000E,520.0,33.56019,-81.92848,29646,34.169781,-82.15474
7061835,2607.0,02010052000E,520.0,33.56019,-81.92848,29142,33.462378,-80.50903
7061836,2607.0,02010052000E,520.0,33.56019,-81.92848,29449,32.715745,-80.26738


In [104]:
station_zip_xref['distance_km'] = haversine_np(
    station_zip_xref.station_long.values,
    station_zip_xref.station_lat.values,
    station_zip_xref.zip_long.values,
    station_zip_xref.zip_lat.values
)

In [105]:
station_zip_xref

,station_id,route_identifier,route_number,station_lat,station_long,Zip,zip_lat,zip_long,distance_km
0,100.0,01040018500N,185.0,34.37925,-82.44926,29607,34.825592,-82.34099,50.578796
1,100.0,01040018500N,185.0,34.37925,-82.44926,29164,33.659078,-81.40845,124.877525
2,100.0,01040018500N,185.0,34.37925,-82.44926,29325,34.470115,-81.86761,54.263882
3,100.0,01040018500N,185.0,34.37925,-82.44926,29520,34.688620,-79.92315,233.784632
4,100.0,01040018500N,185.0,34.37925,-82.44926,29615,34.866801,-82.31739,55.504933
...,...,...,...,...,...,...,...,...,...
7061833,2607.0,02010052000E,520.0,33.56019,-81.92848,29592,34.283207,-79.47272,240.272498
7061834,2607.0,02010052000E,520.0,33.56019,-81.92848,29646,34.169781,-82.15474,70.885076
7061835,2607.0,02010052000E,520.0,33.56019,-81.92848,29142,33.462378,-80.50903,131.964450
7061836,2607.0,02010052000E,520.0,33.56019,-81.92848,29449,32.715745,-80.26738,180.818763


In [1]:
station_zip_xref \
    .loc[station_zip_xref.distance_km <= 50, ['Zip', 'distance_km', 'station_id', 'route_identifier', 'route_number']] \
    .merge(merged_sales[['ZipCode', 'Year', 'Sales']], left_on=['Zip'], right_on=['ZipCode']) \
    .groupby(['station_id', 'route_identifier', 'route_number', 'Year']) \
    .Sales \
    .sum()

NameError: name 'station_zip_xref' is not defined

At this point I can calculate the home sales within a certain radius of a station for the current year or previous year and see if that has any bearing on traffic patterns.

In [98]:
def calc_home_sales(traffic_df, sales_df, zip_xref, radius_cutoff, curr_or_prev='curr'):
    filtered_zips = zip_xref.loc[zip_xref.distance_km <= radius_cutoff]
    sales_modified = sales_df.copy(deep=True)
    sales_modified['next_year'] = sales_modified.Year + 1
    sales_per_station = zip_xref.merge(sales_modified, how='left', left_on='Zip', right_on='ZipCode')
    sales_per_station = sales_per_station.groupby(['station_id', 'route_identifier', 'route_number'])
    
    

,ZipCode,Year,Sales,City,State,Latitude,Longitude
12,29001,2008,10.0,Alcolu,SC,33.769930,-80.17278
13,29001,2009,10.0,Alcolu,SC,33.769930,-80.17278
14,29001,2010,11.0,Alcolu,SC,33.769930,-80.17278
15,29001,2011,8.0,Alcolu,SC,33.769930,-80.17278
16,29001,2012,9.0,Alcolu,SC,33.769930,-80.17278
...,...,...,...,...,...,...,...
4507,29945,2015,26.0,Yemassee,SC,32.681058,-80.83348
4508,29945,2016,14.0,Yemassee,SC,32.681058,-80.83348
4509,29945,2017,23.0,Yemassee,SC,32.681058,-80.83348
4510,29945,2018,17.0,Yemassee,SC,32.681058,-80.83348


In [100]:
merged_sales, station_zip_xref

(      ZipCode  Year  Sales      City State   Latitude  Longitude
 12      29001  2008   10.0    Alcolu    SC  33.769930  -80.17278
 13      29001  2009   10.0    Alcolu    SC  33.769930  -80.17278
 14      29001  2010   11.0    Alcolu    SC  33.769930  -80.17278
 15      29001  2011    8.0    Alcolu    SC  33.769930  -80.17278
 16      29001  2012    9.0    Alcolu    SC  33.769930  -80.17278
 ...       ...   ...    ...       ...   ...        ...        ...
 4507    29945  2015   26.0  Yemassee    SC  32.681058  -80.83348
 4508    29945  2016   14.0  Yemassee    SC  32.681058  -80.83348
 4509    29945  2017   23.0  Yemassee    SC  32.681058  -80.83348
 4510    29945  2018   17.0  Yemassee    SC  32.681058  -80.83348
 4511    29945  2019    3.0  Yemassee    SC  32.681058  -80.83348
 
 [4500 rows x 7 columns],
          station_id route_identifier  route_number  station_lat  station_long  \
 0             100.0     01040018500N         185.0     34.37925     -82.44926   
 1             1